# This script is intended to inspect a Bump test and it compares the measured forces with the stiffness matrix.

In [ ]:
import lsst_efd_client
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
import yaml

In [ ]:
async def get_merge_query(
    efd_client: lsst_efd_client.EfdClient,
    topics: list,
    start_time: Time,
    end_time: Time,
    tolerance=None,
    direction: str = None,
) -> pd.DataFrame:
    """Function to merge multiple topics into a single dataframe.

    Args:
        efd_client (lsst_efd_client.EfdClient): EFD client object.
        topics (list): EFD topics to query and merge.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
        tolerance (float, optional): Tolerance time for matching, see pandas.merge_asof for more information. Defaults to None will use defualt value.
        direction (str, optional): Join method to pass to pd.merge_asod method. Defaults to None will use default value.

    Returns:
        pd.DataFrame: Merged dataframe.
    """
    query_df = list()
    for topic in topics:
        topic_fields = await efd_client.get_fields(topic)
        query = efd_client.build_time_range_query(
            topic, topic_fields, start_time, end_time
        )
        data_query = await efd_client.influx_client.query(query)

        if len(data_query) == 0:
            print(f"{topic} is not present.")
        else:
            data_query.rename(
                columns={
                    col: f"{col}_{topic.split('.')[-1]}"
                    for col in data_query.columns
                },
                inplace=True,
            )
            query_df.append(data_query)

    if len(query_df) == 1:
        return query_df[0]
    elif len(query_df) == 0:
        print("No Dataframe retrieve")
        return None

    query_df.sort(key=lambda el: len(el), reverse=False)
    merge_df = query_df[0].copy()

    for i in range(1, len(query_df)):
        col_left = f"merged_{i}"
        col_right = f"eval_{i}"

        if tolerance is None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                query_df[i],
                direction="nearest",
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

        elif tolerance is None and direction is not None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                query_df[i],
                direction=direction,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

        elif tolerance is not None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                query_df[i],
                direction="nearest",
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
        else:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                query_df[i],
                direction=direction,
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )

    return merge_df

In [ ]:
def get_residual(
    query_df: pd.DataFrame,
    moved_act: list,
    act_movement: dict,
    stiff_matrix: np.array,
    which: str = "axial",
):
    """Generate the residual of each actuator w.r.t to the nominal value in the stiff matrix.

    Args:
        query_df (pd.DataFrame): DataFrame with the bump test data.
        moved_act (list): List of moved actuators.
        act_movement (dict): Dictionary with the start and end times of the actuator movement.
        stiff_matrix (np.array): Stiffness matrix of the mirror.
        which (str, optional): Which actuators to work on (axial or tangent). Defaults to "axial".

    Returns:
        residual (list): residual of each actuator w.r.t to the nominal value
    """

    p = True
    residual = []
    for i, m_act in enumerate(moved_act):
        if which == "axial":
            stiff_to_use = stiff_matrix[int(m_act), :72]
        else:
            stiff_to_use = stiff_matrix[int(m_act) + 72, 72:]

        idi = query_df.loc[
            act_movement[list(act_movement.keys())[i]][0][0]
        ].name
        idi = query_df.index.to_list().index(idi)
        idi = query_df.index[idi + 15]

        idf = query_df.loc[
            act_movement[list(act_movement.keys())[i]][-1][-1]
        ].name
        idf = query_df.index.to_list().index(idf)
        idf = query_df.index[idf - 10]

        if p:
            fig, axs = plt.subplots(
                nrows=2,
                ncols=1,
                layout="constrained",
                subplot_kw={"xlabel": "Time, UTC"},
            )
            query_df.loc[
                act_movement[list(act_movement.keys())[i]][0][
                    0
                ] : act_movement[list(act_movement.keys())[i]][-1][-1],
                f"steps{i}_{which}ActuatorSteps",
            ].plot(ax=axs[0], title="Actuator step", marker="+", color="blue")
            query_df.loc[
                act_movement[list(act_movement.keys())[i]][0][
                    0
                ] : act_movement[list(act_movement.keys())[i]][-1][-1],
                f"measured{i}_{which}Force",
            ].plot(
                ax=axs[1], title="Measured Force", marker="+", color="orange"
            )

            axs[0].axvline(idi, color="r")
            axs[0].axvline(idf, color="r")
            axs[1].axvline(idi, color="r")
            axs[1].axvline(idf, color="r")

            axs[0].set_ylabel("Steps")
            axs[1].set_ylabel("Force, N")

            p = False

        residual.append(
            (
                stiff_to_use
                * (
                    query_df.loc[
                        idi,
                        sorted(
                            [
                                col
                                for col in query_df.columns
                                if "steps" in col
                            ],
                            key=lambda x: int(
                                x.split("_")[0].replace("steps", "")
                            ),
                        ),
                    ].values
                    - query_df.loc[
                        idf,
                        sorted(
                            [
                                col
                                for col in query_df.columns
                                if "steps" in col
                            ],
                            key=lambda x: int(
                                x.split("_")[0].replace("steps", "")
                            ),
                        ),
                    ].values
                )
                - (
                    query_df.loc[
                        idi,
                        sorted(
                            [
                                col
                                for col in query_df.columns
                                if "measured" in col
                            ],
                            key=lambda x: int(
                                x.split("_")[0].replace("measured", "")
                            ),
                        ),
                    ].values
                    - query_df.loc[
                        idf,
                        sorted(
                            [
                                col
                                for col in query_df.columns
                                if "measured" in col
                            ],
                            key=lambda x: int(
                                x.split("_")[0].replace("measured", "")
                            ),
                        ),
                    ].values
                )
            )
        )

    return residual

In [ ]:
def get_moved_actuator(query_df: pd.DataFrame, which: str = "axial"):
    """Get the moved actuators and the time interval of the movement.

    Args:
        query_df: DataFrame with the bump test data.

    Returns:
        moved_act: List of moved actuators.
        act_movement: Dictionary with the start and end times of the actuator movement.
        which (str, optional): Which actuators to work on (axial or tangent). Defaults to "axial".
    """
    which_force = "axialForce" if which == "axial" else "tangentForce"
    moved_act = []
    for col in query_df.columns:
        if (
            "applied" in col
            and which_force in col
            and query_df[col].any() != 0
        ):
            moved_act.append(
                int(col.replace("applied", "").replace(f"_{which_force}", ""))
            )

    moved_act = sorted(moved_act)
    print("Bumped Actuators: ", *moved_act)
    act_movement = {k: [] for k in moved_act}
    for act in moved_act:
        tg_ser = query_df[f"applied{act}_{which_force}"]
        c = tg_ser.index[0]
        for i, row in tg_ser.items():
            if i <= c:
                continue
            if row != 0:
                start = i
                for ii, rrow in tg_ser.loc[i:].items():
                    if rrow == 0:
                        end = ii
                        c = ii
                        break
                act_movement[act].append((start, end))
        if len(act_movement[act]) > 2:
            act_movement[act] = act_movement[act][0:2]
    return moved_act, act_movement

In [ ]:
def plot_residual_map(
    ax: plt.Axes, cell_geom: np.array, residual: list, m_act: int
):
    """Plot the residual map.

    Args:
        fig (plt.Figure): Matplotlib figure object.
        cell_geom (np.array): file containing the position of each atuator in the cell
        residual (list): residual of each actuator w.r.t to the nominal value
        m_act (int): id of the moved actuator
    """

    act_map = [(x, y, val) for (x, y), val in zip(cell_geom, residual)]
    list_res = list(residual)

    vmin = min(list_res)
    vmax = max(list_res)

    ax.set_title(f"#{m_act}")
    ax.scatter(
        [el[0] for el in act_map],
        [el[1] for el in act_map],
        c=[el[2] for el in act_map],
        cmap="coolwarm",
        vmin=vmin,
        vmax=vmax,
    )
    ax.set_xlabel("X (M)")
    ax.set_ylabel("Y (M)")

    cbar = ax.figure.colorbar(
        ax.collections[0],
        ax=ax,
        orientation="vertical",
        label=r"$\Delta$ Force (N)",
    )

In [ ]:
# Initialize the EFD client and upload the cell geometry for plotting.
efd = lsst_efd_client.EfdClient("usdf_efd")

cell_geom = np.array(
    yaml.safe_load(
        open(
            r"C:\Users\utente\Desktop\PhD\LSST\M2\bending_modes\cell_geom.yaml"
        )
    )["locAct_axial"]
)

# Axial Actuators

In [ ]:
stiff_matrix = np.array(
    yaml.safe_load(
        open(
            r"C:\Users\utente\Desktop\PhD\LSST\M2\force_balance\stiff_matrix_surrogate.yaml"
        )
    )["stiff"]
)

topics = [
    "lsst.sal.MTM2.axialActuatorSteps",
    "lsst.sal.MTM2.axialForce",
]

In [ ]:
# Bump 100N (surrogate)
# time_start = Time("2024-06-24T18:54:00")
# time_stop = Time("2024-06-24T21:21:00")

# Bump 10N no tangent (surrogate)
# time_start = Time("2023-08-30T17:23:00")
# time_stop = Time("2023-08-30T22:18:00")

# Bump 10N with tangent (glass)
time_start = Time("2024-07-15T16:27:40")
time_stop = Time("2024-07-15T17:08:30")

stiff_matrix = np.array(
    yaml.safe_load(
        open(
            r"C:\Users\utente\Desktop\PhD\LSST\M2\force_balance\stiff_matrix_m2.yaml"
        )
    )["stiff"]
)

query_df = await get_merge_query(efd, topics, time_start, time_stop)

In [ ]:
moved_act, act_movement = get_moved_actuator(query_df, which="axial")

In [ ]:
# A diagnostic plot on when the measurement are taken will be displayed.
residual = get_residual(
    query_df, moved_act, act_movement, stiff_matrix, which="axial"
)

## Plot the residuals statistic (mean and PtV) over all the 72 axial actuators

In [ ]:
residual_mean = np.array(residual).mean(axis=1)
residual_ptv = np.array(residual).ptp(axis=1)

residual_mean[[5, 15, 25]] = np.nan  # Removing the hardpoints
residual_ptv[[5, 15, 25]] = np.nan  # Removing the hardpoints


fig, axs = plt.subplots(
    nrows=2,
    ncols=1,
    layout="constrained",
    subplot_kw={"xlabel": "Actuator ID"},
)

axs[0].plot(
    moved_act,
    residual_mean,
    marker="x",
    ls="--",
)
axs[0].axvline(x=30.5, color="green", label="B-ring")
axs[0].axvline(x=54.5, color="orange", label="C-ring")
axs[0].axvline(x=72, color="red", label="D-ring")
axs[0].set_ylabel("Mean Measured - Stiff, N")

axs[1].plot(
    moved_act,
    residual_ptv,
    marker="x",
    ls="--",
)
axs[1].axvline(x=30.5, color="green", label="B-ring")
axs[1].axvline(x=54.5, color="orange", label="C-ring")
axs[1].axvline(x=72, color="red", label="D-ring")


axs[1].set_ylabel("PtV Measured - Stiff, N")

handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles=handles, labels=labels, loc="upper right")

In [ ]:
fig = plt.figure(layout="constrained", figsize=(20, 25))
for a, val in act_movement.items():
    ax = fig.add_subplot(12, 6, a + 1)
    ax.set_box_aspect(1)
    plot_residual_map(ax, cell_geom, residual[a], moved_act[a])

# Tangent Link

In [ ]:
topics = [
    "lsst.sal.MTM2.tangentActuatorSteps",
    "lsst.sal.MTM2.tangentForce",
]

# Bump 10N with tangent
time_start = Time("2024-07-15T16:27:40")
time_stop = Time("2024-07-15T17:08:30")

query_df = await get_merge_query(efd, topics, time_start, time_stop)

In [ ]:
tangent_ang = np.array(
    yaml.safe_load(
        open(
            r"C:\Users\utente\Desktop\PhD\LSST\M2\bending_modes\cell_geom.yaml"
        )
    )["locAct_tangent"]
)

tangent_rad = yaml.safe_load(
    open(r"C:\Users\utente\Desktop\PhD\LSST\M2\bending_modes\cell_geom.yaml")
)["radiusActTangent"]

cell_geom = [
    (
        np.sin(np.deg2rad(ang)) * tangent_rad,
        np.cos(np.deg2rad(ang)) * tangent_rad,
    )
    for ang in tangent_ang
]

In [ ]:
moved_act, act_movement = get_moved_actuator(query_df, which="tangent")
residual = get_residual(
    query_df, moved_act, act_movement, stiff_matrix, which="tangent"
)

In [ ]:
residual_mean = np.array(residual).mean(axis=1)
residual_ptv = np.array(residual).ptp(axis=1)


fig, axs = plt.subplots(
    nrows=2,
    ncols=1,
    layout="constrained",
    subplot_kw={"xlabel": "Actuator ID"},
)

axs[0].plot(
    moved_act,
    residual_mean,
    marker="x",
    ls="--",
)
axs[0].set_ylabel("Mean Measured - Stiff, N")

axs[1].plot(
    moved_act,
    residual_ptv,
    marker="x",
    ls="--",
)
axs[1].set_ylabel("PtV Measured - Stiff, N")

In [ ]:
fig = plt.figure(layout="constrained", figsize=(12, 6))
for a, val in act_movement.items():
    ax = fig.add_subplot(2, 3, a + 1)
    ax.set_box_aspect(1)
    plot_residual_map(ax, cell_geom, residual[a], moved_act[a])